In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}



<IPython.core.display.Javascript object>

In [11]:
from contextlib import contextmanager
import sys, os
import scipy.linalg as lin
import numpy.linalg as linNum
import numpy as numpy
import itertools 
import copy
from cysignals import init_cysignals

#from sympy import *
from sympy import Matrix
#from sage.interfaces.phc import phc
import phcpy as phc
import ipywidgets as widgets
from sage.plot.plot3d.plot3d import axes
from sage.plot.colors import rainbow
init_cysignals()
numpy.seterr('raise')
numpy.set_printoptions(precision=3,suppress=True,formatter={'float': '{: 0.3f}'.format}) #comment this for debug
# maybe use numpy.negative to negate numbers

   
@contextmanager
def suppress_stdout():#used to suppress output from phcpack
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

class MyError(Exception):
    pass

def clearAnimation(arrayOfPoints,arrayOfPointsC):
   
    arrayOfPoints.clear()
    arrayOfPointsC.clear()
def drawLinesHelper(lines,stateArr,ns,**options):
    resu = []
    preceeding = Graphics()
    #lastLineBeforeGap = ()#line before gap
    if stateArr is None:
        for j in range(1,len(lines)):
            #x1, y1,z1 = None,None,None
            #if not lines[j-1] and not lines[j]:
            #    resu.append(preceeding + Graphics())
            #    preceeding = resu[-1]
            #    continue
            #if lines[j-1] and not lines[j]:
            #    lastLineBeforeGap = lines[j-1]
            #    resu.append(preceeding + Graphics())
            #    preceeding = resu[-1]
            #    continue
            #if not lines[j-1] and not lines[j] and lastLineBeforeGap:
            #    x1, y1,z1 = lastLineBeforeGap
            #if not lines[j-1]:
            #    resu.append(preceeding + Graphics())
            #    preceeding = resu[-1]
            #    continue
            #lastLineBeforeGap = lines[j-1]
            x1, y1,z1 = lines[j-1]
            x2, y2,z2 = lines[j]
        
            dx = float(x2 - x1)/float(ns)
            dy = float(y2 - y1)/float(ns)
            dz = float(z2 - z1)/float(ns)
            for i in range(ns):
                xx = x1 + (i+1)*dx
                yy = y1 + (i+1)*dy
                zz = z1 + (i+1)*dz
                resu.append(preceeding + line([(x1,y1,z1), (xx,yy,zz)], **options))
            preceeding = resu[-1]
            if j == len(lines)-1:
                for i in range(ns):
                    xx = x1 + (i+1)*dx
                    yy = y1 + (i+1)*dy
                    zz = z1 + (i+1)*dz
                    resu.append(preceeding + line([(x2,y2,z2), (xx,yy,zz)], **options))
         
    else:
        i = 0
        j = 0
        while j < len(lines):
            if stateArr[i] == 1:
                resu.append(preceeding+Graphics())
                j += stateArr[i]
                i += 1
                continue
            resu.append(preceeding + line(lines[j:j+stateArr[i]], **options))
            lastline = lines[j:j+stateArr[i]]
            j+= stateArr[i]
            if i  < len(stateArr):
                i+=1;
        preceeding = resu[-1]
        
           
    return resu
@parallel
def drawlines(ns,endIndex,arrayOfPoints, **options):
    r"""
    Input:
    - ``lines``: a list of lines, each line being defined
      by its endpoints as ``[(x1,y1,z1), (x2,y2,z2)]``
    - ``ns``: the number of division of each line for the 
      animation
    - ``**options``: any option to pass to the graphic 
      command ``line``, like ``color`` or ``thickness``
    """
    resu = []
    preceeding = Graphics()
    anim = Graphics()
    colorsDict = dict(zip(arrayOfPoints.keys(),rainbow(len(arrayOfPoints.keys()))))
    stateArr = arrayOfPoints['state']
    arrayOfPoints.pop('state')
    
    for key, word in arrayOfPoints.items():
        #lines = numpy.array_split(word,2)
        #word = [list(ele) for ele in word] #make list of tuples to list of lists
        #lines = word[:endIndex]#was lines[:endIndex]
        if key == 'zp' or key == 'yp':
            resu = drawLinesHelper(word,None,ns, color=colorsDict[key], thickness=2,**options)
            anim += animate(resu).interactive()
        else:
            resu = drawLinesHelper(word,stateArr,ns, color=colorsDict[key], thickness=2,**options)
            anim += animate(resu).interactive()
        #print(len(lines)) #debug?
    return anim
def customAtan(y,x):
    v=numpy.mod(numpy.arctan2(y,x)+ 2*numpy.pi,2*numpy.pi)
    return v
def svdcomposition(b,firstPhi,secondPhi):
    #rf_low = RealField(12)
    #print("product") DEBUG
    #print(ux@sx@vhx)
    #print("ux before numpy svd")
    #print(ux)
    #print("vhx before numpy svd")
    #print(vhx)
    uy = numpy.array([[numpy.cos(firstPhi),-numpy.sin(firstPhi)],[numpy.sin(firstPhi),numpy.cos(firstPhi)]]) #matrix(rf_low,2,2,[cos(firstPhi),-sin(firstPhi),sin(firstPhi),cos(firstPhi)])
    sy = numpy.array([[b,0],[0,1/b]])#matrix(rf_low,2,2,[b,0,0,1/b])
    vhy = numpy.array([[numpy.cos(secondPhi),-numpy.sin(secondPhi)],[numpy.sin(secondPhi),numpy.cos(secondPhi)]]) #matrix(rf_low,2,2,[cos(secondPhi),-sin(secondPhi),sin(secondPhi),cos(secondPhi)])
    return uy@sy@vhy

    
def makeunitarydeterminant1(u,v,msign,s):
    """
    v means the V transpose matrix
    msign is the sign of the matrix
    diagonal matrix s
     ---------
     | a | b |
     | c | d | 
     ---------
     
     make ad-bc = 0
    """
    
    negativePivotLead = numpy.array([[-1,0],[0,1]])#matrix(RealField(12),2,2,[-1,0,0,1])
    negativePivotFollow = numpy.array([[1,0],[0,-1]])#matrix(RealField(12),2,2,[1,0,0,-1])
    
    top = negativePivotLead @ v
    bottom = negativePivotFollow @ v
    left =   u @negativePivotLead
    right = u @ negativePivotFollow 
    leftRight =  u @negativePivotLead @ negativePivotFollow
    topBottom =   negativePivotLead @ negativePivotFollow @ v
    
    array = numpy.array([[linNum.det(left),linNum.det(right),linNum.det(right),linNum.det(left),linNum.det(left),linNum.det(right),linNum.det(leftRight),linNum.det(leftRight),linNum.det(leftRight)],[linNum.det(top),linNum.det(bottom),linNum.det(topBottom),linNum.det(top),linNum.det(bottom),linNum.det(topBottom),linNum.det(topBottom),linNum.det(top),linNum.det(bottom)]])
    array = numpy.sum(array,axis=0)
    arrayMatrixRotation = numpy.array([[left,right,leftRight,right,left,left,right,leftRight,leftRight],[s,s,s,s,s,s,s,s,s],[top,bottom,topBottom,top,bottom,topBottom,topBottom,top,bottom]])
    #arrayProduct = numpy.prod(arrayMatrixRotation,axis=0)#matrix multiplication gives differing result than @
    #if (numpy.sign(u[0,0]) == 1 and numpy.sign(u[0,1]) == 1 and numpy.sign(u[1,0]) == 1 and numpy.sign(u[1,1])== -1) or  (numpy.sign(u[0,0]) == -1 and numpy.sign(u[0,1]) == 1 and numpy.sign(u[1,1]) == 1 and numpy.sign(u[1,0]) == 1) :
    #    array[0] = 0
    if(numpy.sign(u[1,0]) == 1 and (math.isclose(numpy.abs(u[0,0]),1,abs_tol=1e-16) == False)): #isclose is used when value is close to 1
        array[0] = 0
    if(numpy.sign(u[0,0]) == 1 and (math.isclose(numpy.abs(u[0,0]),1,abs_tol=1e-16))): 
        array[0] = 0
        
    arrayProduct = []
    boolArray = []
    for i in range(arrayMatrixRotation.shape[1]):
        arrayProduct.append(arrayMatrixRotation[0,i] @ arrayMatrixRotation[1,i] @ arrayMatrixRotation[2,i])
        
    for elem in arrayProduct:
        #print("elem\n")
        #print(numpy.sign(elem))
        #print("msign\n")
        #print(msign)
        if numpy.array_equal(numpy.sign(elem),msign):
            boolArray.append(True);
        else:
            boolArray.append(False);
    
    max_index = array.argmax()
 
   # print(msign) #debug
    #print(numpy.sign(arrayProduct))
    #print(array)
    #print(boolArray)
    if(max_index == 0 and boolArray[0] == True):
        v = top
        u = left
    elif(max_index == 1 and boolArray[1] == True):
        v = bottom
        u = right
    elif(max_index == 2 and boolArray[2] == True):
        v = top
        u = right
    elif(max_index == 3 and boolArray[3] == True):
        v = bottom
        u = left
    elif(max_index == 4 and boolArray[4] == True):
        v = topBottom
        u = left
    elif(max_index == 5 and boolArray[5] == True):
        v = topBotom
        u = right
    elif(max_index == 6 and boolArray[6] == True):
        v = top
        u = leftRight
    elif(max_index == 7 and boolArray[7] == True):
        v = bottom
        u = leftRight
    elif(max_index == 8 and boolArray[8] == True):
        v = topBottom
        u = leftRight
    
       
       
  
        
    return u,v;
p = 0
def getDistances(sols,stepsize):
    global p
   # allsets = chain(*map(lambda x: combinations(cleanSols, x), range(0, len(cleanSols)+1)))
    pointPermutations = []
    distances = []
    #print(sols)#debug
    #sols = [sorted(frame, key=lambda x: linNum.det(x)) for frame in sols] #sort each frame before? #det may not work if two points happen to have same determinant
    #print(sols)
    for j in range(1,len(sols)):
            firstPoints = [(sublist,idx) for idx,sublist in enumerate(sols[j-1],1)] #for item in sublist for x in item]
            secondPoints = [(sublist,idx) for idx,sublist in enumerate(sols[j],1)] #for item in sublist for x in item]
            #print(firstPoints) debug
            #print(secondPoints) debug
            pointPermutations.append([*itertools.product(firstPoints,secondPoints)])
            #print(*itertools.product(firstPoints,secondPoints))
            #print('neww')
    #pointPermutations.append([*itertools.product([(sublist,idx) for idx,sublist in enumerate(sols[0],1)],[(sublist,idx) for idx,sublist in enumerate(sols[len(sols)-1],1)])])#the last permutation is the last frame and the first frame to see if they complete
   # print(len(pointPermutations))
    for permutation in pointPermutations:
        permutationArray = []
        for pair in permutation:
            #print(pair) #debug
            #print(pair[0][0][0][0])
            #print(pair[1][0][0][0])
            #print(pair[0][0][0][1])
            #print(pair[1][0][0][1])
            #print(pair[0][0][1][0])
            #print(pair[1][0][1][0])
            #print(pair[0][0][1][1])
            #print(pair[1][0][1][1])
            pairdist = (
          math.sqrt((pair[0][0][0][0].real-pair[1][0][0][0].real)**2+(pair[0][0][0][0].imag-pair[1][0][0][0].imag)**2)
        + math.sqrt((pair[0][0][0][1].real-pair[1][0][0][1].real)**2+(pair[0][0][0][1].imag-pair[1][0][0][1].imag)**2)
        + math.sqrt((pair[0][0][1][0].real-pair[1][0][1][0].real)**2+(pair[0][0][1][0].imag-pair[1][0][1][0].imag)**2)
        + math.sqrt((pair[0][0][1][1].real-pair[1][0][1][1].real)**2+(pair[0][0][1][1].imag-pair[1][0][1][1].imag)**2),(pair[0][1],pair[1][1]))
            permutationArray.append(pairdist)
        distances.append(permutationArray)
    distancesSorted = []
    #print(distances)#debug
    #print('next')
    for frame in distances:
       # frame= [i for i in frame if i[0] <= 1]
        frame = sorted(frame, key=lambda v: v[0])
        
        visited_data = set()
        frameArray = {}
        for a, b in frame:
            if not b[0] in visited_data:
                visited_data.add(b[0])
                frameArray.setdefault((a,b[0]),[]).append(list(b))
                #frameArray.append((a, b))
            elif b[0] in visited_data and (a,b[0]) in frameArray:
                frameArray[(a,b[0])].append(list(b))
            
        
        distancesSorted.append(list(frameArray.values()))
    #print(distancesSorted) #debug
    distancesSorted = [sorted(sublist, key=lambda x: x[0][0]) for sublist in distancesSorted]
    
    for i in range(1,len(sols)):
        listToInsert = []
        
        pairDictP = {}
        pairDeepP = []
        pairDictN = {}
        pairDeepN = []
        if i != len(sols) -1:
            for s in range(0,len(distancesSorted[i])):
                for t in range(0,len(distancesSorted[i][s])):
                    pairDictP.setdefault(distancesSorted[i][s][t][0],[]).append((s,t))
            pairDeepP = copy.deepcopy(distancesSorted[i])
            for s in range(0,len(distancesSorted[i-1])):
                for t in range(0,len(distancesSorted[i-1][s])):
                    pairDictN.setdefault(distancesSorted[i-1][s][t][1],[]).append((s,t))

            pairDeepN = copy.deepcopy(distancesSorted[i-1])
        for j in range(0,len(distancesSorted[i-1])):
            #sols[i][distancesSorted[j][0]-1], sols[i][distancesSorted[j][1]-1] = sols[i][distancesSorted[j][1]-1], sols[i][distancesSorted[j][0]-1]
            #listToInsert.insert(distancesSorted[i-1][j][1]-1,sols[i][distancesSorted[i-1][j][0]-1])
            if i != len(sols) -1:
               # print("i-1")
               # print(distancesSorted[i-1])
               # print("i")
               # print(distancesSorted[i])
                #try:
                distancesSorted[i][j].clear()
                
                for e,y in pairDictP[j+1]:
                    if j+1 not in pairDictN:
                        distancesSorted[i][j].append([pairDeepP[e][y][0],pairDeepP[e][y][1]])
                        continue
                    for w,v in pairDictN[j+1]:
                        if [pairDeepN[w][v][0],pairDeepP[e][y][1]] not in distancesSorted[i][j]: #remove duplicates
                            distancesSorted[i][j].append([pairDeepN[w][v][0],pairDeepP[e][y][1]])
                    #distancesSorted[i][distancesSorted[i-1][j][l][1]-1].extend()
                
                
               # for l in range(0,len(distancesSorted[i-1][j])):
                    
                   # if l < len(distancesSorted[i][j]):
                     #   distancesSorted[i][distancesSorted[i-1][j][l][1]-1][l][0] = distancesSorted[i-1][j][l][0]
        
                            
                    #else:
                     #   distancesSorted[i][distancesSorted[i-1][j][l][1]-1].append([distancesSorted[i-1][j][l][0],distancesSorted[i][distancesSorted[i-1][j][l][1]-1][0][1]])
                        #distancesSorted[i][distancesSorted[i-1][j][l][1]-1].extend()
                    #distancesSorted[i][distancesSorted[i-1][j][1]-1][0] = distancesSorted[i-1][j][0]
                #except IndexError:
                #    print("Index skipped"+str(i)+" "+str(j))
                #    pass
               # print("after i -1")
               # print(distancesSorted[i])
                #distancesSorted[i-1]
               # print("after i")
               # print(distancesSorted[i])
                
                
            
       #distancesSorted = [sorted(sublist, key=lambda x: x[0][0]) for sublist in distancesSorted]
       
        #print('ok')
        #print(distancesSorted)
        #print('next')
        #sols[i] = listToInsert;
   
    #print(distancesSorted)
    #print(sols)
    return distancesSorted[-1]
    #p+=1
    #if(p == 1):
    #    getDistances(sols,stepsize)
    #print(len(distances))
    #print(distances)
    #distances = list(filter(lambda x: x[0]<=stepsize, distances)) #follow points
    #print(distances)#debug
    #swapIndexes = list(zip(*distances))[1]
    #tempArr = []
    #print(swapIndexes)
    #for i in range(0,len(swapIndexes),16):
    #    tempArr.append(swapIndexes[i:16+i])
        
    #swapIndexes = tempArr
    #print(swapIndexes)
    

    
def findRoots(wordMatrix,zM,word,arrayOfPointsC,printWord=False):
    r"""
    Solve for X matrix
    """
    
    system = [str(wordMatrix[0,0]-zM[0,0])+';', str(wordMatrix[0,1]-zM[0,1])+';',str(wordMatrix[1,0]-zM[1,0])+';',str(wordMatrix[1,1]-zM[1,1])+';']
    #print(system)#debug
    phc.phcpy2c3.py2c_set_seed(2013)
    (q, c) = phc.solver.standard_scale_system(system)
    with suppress_stdout():
        v=phc.solver.solve(q,mvfocus=1)#v=phc.solver.solve(system,mvfocus=1)
    v = phc.solver.standard_scale_solutions(len(q), v, c)
    sols = []
    alsols = []
    if v is None:
        return None
    #v.sort();
    #v[1], v[2]  = v[2], v[1]
    
    for sol in v:
        alsols.append(phc.solutions.strsol2dict(sol))
        realsols = phc.solutions.is_real(sol,0.01) #true if solutions are real within a tolerance
        if realsols:
            sols.append(phc.solutions.strsol2dict(sol))
            if printWord:
                print(sol)
    if not sols:
        return None
    if not printWord:
        arrayOfPointsC.setdefault(word,[]).append([[[x['x11'],x['x12']],[x['x21'],x['x22']]] for x in alsols ])
    return sols
def retrieveWords(w,xM,yM,zM,arrayOfPointsC,printWord=False,makecorrection=False):
    
    wordArray = w.split(",")
  
    wordArray = list(filter(lambda x: (''.join(x.split())),wordArray))
   
    wordArray = list(filter(lambda x: (set(x) - set('xyXY') == set()), wordArray))
    wordArray = list(set(wordArray))
    
    wordTuples = []
   
    hasX = False;#phcpack won't accept if there are no variables
    for word in wordArray:
        wordMatrix = numpy.array([[1,0],[0,1]])#matrix(RealField(12),2,2,[1,0,0,1])##
        for character in word:
            if character == 'x':  
                wordMatrix = wordMatrix @ xM
                hasX=True
            elif character == 'y': 
                wordMatrix = wordMatrix @ yM
            elif character == 'X':
                wordMatrix = wordMatrix @ numpy.array(Matrix(xM).inv())#xM.inverse()#linNum.inv(xM)
            elif character == 'Y':  
                wordMatrix = wordMatrix @ numpy.array(Matrix(yM).inv())#yM.inverse()#linNum.inv(yM)
            else:
                raise MyError("Unrecognized Character");
        #print(wordMatrix) debug
        if not hasX:
            continue;
        if printWord:
            #display(widgets.HTMLMath("%s = z_{0} := %s = %s$"%(r"$"+word, latex(matrix(wordMatrix.tolist())),latex(matrix(zM.tolist())))))
         
            display(widgets.HTMLMath("<br />%s = z_{0} := %s = %s$"%(r"$"+word, latex(matrix(wordMatrix.tolist())),latex(matrix(zM.tolist())))))
            
        wordMatrix = findRoots(wordMatrix,zM,word,arrayOfPointsC,printWord)
        if wordMatrix is None:#check if complex
            continue;
        for idx,elem in enumerate(wordMatrix,1):
            #print("elems")
            #print(elem)
            #print("done")
            element = numpy.array([[elem['x11'].real,elem['x12'].real],[elem['x21'].real,elem['x22'].real]])
            u,s,vh = lin.svd(element,lapack_driver='gesvd')#lin.svd(wordMatrix.numpy())#use scip.svd or numpy.svd
            s = numpy.diag(s)#make s diagonal
            #print("before u") #DEBUG
            #print(u)
            #print("before vh")
            #print(vh)
            if makecorrection:
                u,vh = makeunitarydeterminant1(u,vh,numpy.sign(element),s)
            #print("after u")
            #print(u)#test
            #print("after vh")
            #print(vh)#test
            #vh = makeunitarydeterminant1(vh)
            wordTuples.append((word,s[0,0],(customAtan(u[1][0],u[0][0])),customAtan(vh[1][0],vh[0][0]),u[1][0],u[0][0],vh[1][0],vh[0][0]))
    
    return wordTuples
def build_frame(b,firstTheta,secondTheta,c,firstPhi,secondPhi,w,arrayOfPoints,arrayOfPointsC,makecorrection=False):
    yM=svdcomposition(b,firstTheta,secondTheta)
    zM=svdcomposition(c,firstPhi,secondPhi)
    M = Manifold(3, 'R^3', r'\RR^3', structure='topological', start_index=0)
    spher.<r,th,ph1> = M.chart(r'r:(0,+oo) th:(0,2*pi):periodic:\theta ph1:(0,2*pi):periodic:\phi_1')
    cart.<x,y,z>=M.chart()
    spher.transition_map(cart,[th,ph1,r])
   
    y = M.point((b,firstTheta,secondTheta), chart=spher,name = 'y')
    arrayOfPoints.setdefault('yp',[]).append((firstTheta,secondTheta,b))
    z = M.point((c,firstPhi,secondPhi), chart=spher,name = 'z')
    arrayOfPoints.setdefault('zp',[]).append((firstPhi,secondPhi,c))
    R.<x11,x12,x21,x22> = PolynomialRing(RDF,4)
    xM = numpy.array([[x11,x12],[x21,x22]])
    G = y.plot(cart,color = 'green')+z.plot(cart, color = 'orange')+axes(1,0.5,color='black')
    wordsArray = retrieveWords(w,xM,yM,zM,arrayOfPointsC,False,makecorrection)
       
    #plot words
    if wordsArray is not None:
        arrayOfPoints.setdefault("state",[]).append(len(wordsArray)) 
        for word,d,firstPsi,secondPsi,fy,fx,sy,sx in wordsArray:
            G += M.point((d,firstPsi,secondPsi),chart = spher, name = word).plot(cart,color = 'red')
            arrayOfPoints.setdefault(word,[])
            #ypLen = len(arrayOfPoints['yp'])
            #if(len(arrayOfPoints[word]) == 0 and ypLen > 1 ):
            #    for i in range(ypLen -1):
            #        arrayOfPoints[word].append(())
            arrayOfPoints[word].append((firstPsi,secondPsi,d))
            
       
            #print("First Theta Error:(firsttheta-firstPsi) " + str(firstTheta - firstPsi) +" 1the " + str(firstTheta) + " 1psi " + str(firstPsi) + " y " + str(fy) + " x " +str(fx))
            #print("Second Theta Error " + str(secondTheta - secondPsi) +" " + str(secondTheta) + " " + str(secondPsi) + " " + str(fy) + " " +str(fx))
        #set1,set2  = set(arrayOfPoints.keys()),set([x[0] for x in wordsArray])
        
        #set3 = set1 - set2 #1 
      
        #print("set1"+str(set1))#Debug
        #print("set2"+str(set2))#Debug
        #print(set3)#debug
        #set3.remove('yp') #1
        #set3.remove('zp') #1
        #set3.remove('state')
        #for word in set3:
        #    arrayOfPoints[word].append(())#fill gaps in animation if exists
        
            
    return G
                                

    
    

    
#first angle is [0,pi) and second angle is [0,2*pi]. Scalar(a,b) is greater than 0
@interact(layout=dict(top=[['w']],left=[['b'],['firstTheta'],['secondTheta']],right=[['c','firstPhi','secondPhi']],bottom=[['makecorrection']]))



def main(w='xyx',b=slider(1.1,100,0.1,2,label='$b_{0}$'),firstTheta=slider(0,3.14,0.01,1.54,label=r'\(\theta_1\)'),secondTheta=slider(0,2*3.14,0.01,2.30,label=r'\(\theta_2\)'),c=slider(1.1,100,0.1,2,label='$c_{0}$'),firstPhi=slider(0,3.14,0.01,2.90,label=r'\(\phi_1\)'),secondPhi=slider(0,2*3.14,0.01,4.87,label=r'\(\phi_2\)'),makecorrection=False,auto_update=False):


    
    arrayOfPoints = {}#used for animation, the order of words is by first in line
    arrayOfPointsC = {}#points by word complex
   
    yM=svdcomposition(b,firstTheta,secondTheta)
    zM=svdcomposition(c,firstPhi,secondPhi)
    #pretty_print(html("$y = %s$  $z= %s$"%(latex(matrix(zM.tolist())), latex(matrix(yM.tolist())))))
   # pretty_print(html("Determinant  is   %s"%(latex(d)))) #uncomment to print determinant

    M = Manifold(3, 'R^3', r'\RR^3', structure='topological', start_index=0)
    spher.<r,th,ph1> = M.chart(r'r:(0,+oo) th:(0,2*pi):periodic:\theta ph1:(0,2*pi):periodic:\phi_1')
    cart.<x,y,z>=M.chart()
    spher.transition_map(cart,[th,ph1,r])

    #print('x from input') #debug
    #print(a)
    #print(firstTheta)
    #print(secondTheta)
    #print('y from input')
    #print(b)
    #print(firstPhi)
    #print(secondPhi)

    yPoint = M.point((b,firstTheta,secondTheta), chart=spher,name = 'y0')
    zPoint = M.point((c,firstPhi,secondPhi), chart=spher,name = 'z0')
    R.<x11,x12,x21,x22> = PolynomialRing(RDF,4)
    xM = numpy.array([[x11,x12],[x21,x22]])
    wordsArray = retrieveWords(w,xM,yM,zM,arrayOfPointsC,True,makecorrection)
    G = yPoint.plot(cart,color = 'green')+zPoint.plot(cart, color = 'orange')+axes(1,0.5,color='black')
       
    #plot words
    if wordsArray is not None:
         for word,d,firstPsi,secondPsi,fy,fx,sy,sx in wordsArray:
            G += M.point((d,firstPsi,secondPsi),chart = spher, name = word).plot(cart,color = 'red')
            #print(word + " from svd") #debug
            #print(c)
            #print(firstPsi)
            #print(secondPsi)
   

    G.show(axes_label=['phi_1','phi_2','c'])#latex not supported yet for axex_label https://trac.sagemath.org/ticket/30226 todo
    
    display(widgets.HTMLMath('<h3>Animation where %s is not constant'%(r'\(\theta_2\)')))
    #Let's animate!
    @interact(layout=dict(left=[['leftBound']],right=[['rightBound']],bottom=[['stepSize','lines','makecorrection']]))
    def _(leftBound=input_box(0.1),rightBound=input_box(3.1),stepSize=input_box(0.1),lines=True,makecorrection=makecorrection,auto_update=False ):
        frames = [build_frame(b,firstTheta,i,c,firstPhi,secondPhi,w,arrayOfPoints,arrayOfPointsC) for i in (leftBound,leftBound+stepSize,leftBound+stepSize+stepSize,.., rightBound)]# could do for a,b in zip(range1,range2)
      
        #print(arrayOfPoints)#debug
        #for word in arrayOfPoints.keys():#debug print number of points per word
         #   print(len(arrayOfPoints[word]))#debug print number of points per word
        if lines:
            plot = drawlines(1,0,arrayOfPoints) + animate(frames).interactive() #may be slow
        else:
            plot = animate(frames).interactive() #may be slow
        for key, points in arrayOfPointsC.items():
            print(key)
            print([tuple(tup) for tup in getDistances(points,stepSize)])
        plot.show(delay=10, auto_play=False, projection='orthographic',format='mp4')
        clearAnimation(arrayOfPoints,arrayOfPointsC)

    
    #pretty_print(html('<h3>Animation where %s is not constant'%(r'\(\theta_2\)')))
    #frames = [build_frame(b,firstTheta,i,c,firstPhi,secondPhi,w,arrayOfPoints,arrayOfPointsC) for i in (0, 0.2, 0.3, .., 2*numpy.pi)]# could do for a,b in zip(range1,range2)
  
    #plot = animate(frames).interactive() + drawlines(1,0,arrayOfPoints)
    #plot.show( delay=10, auto_play=False, projection='orthographic',format='mp4')
    #arrayOfPoints.clear()#clear points because interactive will not
    #show(drawlines(1,0,arrayOfPoints), delay=10, auto_play=False, projection='orthographic')
    #clear arrayOfPoints after an animation with any of the variables
    #pretty_print(html('<h3>Animation where firstTheta and firstPhi are not constant'))
    #changes below to pi from 2*pi
    #frames = [build_frame(b,j,k,c,firstPhi,secondPhi,w,arrayOfPoints,arrayOfPointsC) for j,k in zip((0.1, 0.2, 0.3, .., numpy.pi),(0.1, 0.2, 0.3, .., numpy.pi))]# could do for a,b in zip(range1,range2)
    #plot = animate(frames).interactive()
    #show(plot, delay=10, auto_play=False, projection='orthographic')

Manual interactive function <function main at 0x7f6babdf4d30> with 8 widgets
  w: Text(value='xyx', descriptio…